At the Pycon 2016 sprints, there were a bunch of folks who wanted to sprint on Bokeh and so needed to setup a development environment.

[Eric Ma](https://github.com/ericmjl) kindly put together a script to help people get started here: https://github.com/bokeh/bokeh/issues/4427 but then new things came to light.

Hopefully this is now a useful guide to getting started developing bokeh.

### Install conda

You don't have to do this, but its the simplest most repeatable way to get everything you need to install bokeh installed in a repeatable way (including non-python packages).

I use miniconda, which is "a minimal distribution of Python containing only Python, the conda package and environment manager, and only the packages needed by Python or conda" [source](https://docs.continuum.io/anaconda/index).

Download your version here: http://conda.pydata.org/miniconda.html

I would recommend the python3.5 version. Note: you don't need python3.5 to run it, it gives you python 3.5 by default.

Also worth noting that although it gives you a distribution of python, it isn't going to eat your system python or virtual env or anything else you have set-up on your local machine. And if you decide you don't like it (hasn't happened yet :D) you should be able to uninstall it without hassle.

While I definitely support people using whatever way they want to get setup with Bokeh, using conda is probably the best way if you want to get help if something goes wrong.

### Set-up your development conda environment 

This is like a virtualenv if you're familiar with them. We create a conda environment called bokeh-dev and we install python 3.4 in it (we use python 3.4 because there are a number of dependencies for bokeh that don't have python 3.5 versions built yet).

    $ conda create -n bokeh-dev python=3.4
Now we activate the environment:

    $ source activate bokeh-dev


### Fork & clone bokeh

If you've never worked on open source before, some of this may be new, and I would recommend 
reading the following guides from github:

* https://help.github.com/articles/fork-a-repo/
* https://help.github.com/articles/working-with-forks/
* https://help.github.com/articles/syncing-a-fork/

Fork the bokeh repo here: https://github.com/bokeh/bokeh and clone your forked copy.

From here on, this tutorial assumes you're in the root of the directory you cloned - something like this:

```sh
(bokeh-dev)➜  bokeh git:(master) ls
CHANGELOG        MANIFEST.in      bin              bokehjs          examples   
scripts          setup.cfg        sphinx           versioneer.py
LICENSE.txt      README.md        bokeh            conda.recipe     requirements.txt 
secrets.tar.enc  setup.py         tests
```

### Install the bokeh dependencies

We have a script for this, but I tried to use it and it isn't quite working (as of June 4, 2016 - I'm working on a PR to fix it). So in the meantime.

    
* Install bokeh core dependencies

```sh
$ conda install nodejs six python-dateutil numpy yaml pyyaml jinja2 tornado requests futures-compat -c bokeh -c javascript
``` 
    
* Install bokeh test, docs and examples dependencies

```sh
$ conda install graphviz pygments sphinx colorama coverage flake8 mock pandas pytest pytest-cov==1.8.1 pytest-selenium pytest-xdist pytest-rerunfailures beautiful-soup scipy pillow boto jupyter matplotlib sympy ggplot seaborn flask flexx -c bokeh
```

### Install the javascript dependencies

We go into the `bokehjs` directory, and use node's npm to install the dependencies

```sh
$ cd bokehjs
$ npm install
```

This may take a while, and afterwards you should see a new `node_modules` directory:

```sh
(bokeh-dev)➜  bokehjs git:(master) ✗ ls
CHANGELOG.md    LICENSE         examples        gulpfile.js     package.json    test
CONTRIBUTING.md README.md       gulp            node_modules    src             typings.json
```

### Let's try building bokeh

We go up one level to the root directory and use setup.py to build bokeh for the first time.

```sh
$ cd ..
$ python setup.py develop --build-js
```

This will take 30s to a minute as it takes a while to build and minify bokehjs.

Hopefully this all went to plan, and now we can try running the tests or running some examples.

### Running the tests

Bokeh has lots of tests. Some of them are not that easy or necessary to run locally, so it's good to be specific. 

To run the python unit tests:

```sh
$ ulimit -n 1024
$ py.test -m 'not (integration or js or examples)' 
```

(The `ulimit` step is because some of our tests open a lot of files to test the server and this makes it work - we should probably add this into our test suite somehow)


To run the javascript unit tests you have two options - directly with the javascript packages or through py.test.

* Through py.test

```sh
$ py.test -m js
```

* With js packages

```sh
$ cd bokehjs
$ ./node_modules/gulp/bin/gulp.js test
```

To run the integration tests:

```sh
$ py.test -m integration
```

The integration tests are selenium tests and so when you want to run them you should see lots of firefox browser windows pop up.

You can speed up any python tests by adding `-n 4` (or any number) to run the tests in parallel over multiple cpu cores.

The keen-eyed will have noticed that we haven't run the examples tests. These require a little more work and aren't really necessary to run locally - especially if you're just getting started with bokeh.

More on testing here: http://bokeh.pydata.org/en/dev/docs/dev_guide/testing.html

### Running the examples

See the [post on running examples](/blog/2016/06/05/running-bokeh-examples/). In particular, go to the end to see how to use your newly built local copy of bokeh [/blog/2016/06/05/running-bokeh-examples#Running-with-locally-installed-bokeh-(advanced)](/blog/2016/06/05/running-bokeh-examples#Running-with-locally-installed-bokeh-(advanced))

### Running the docs


We want to make sure that we use the locally built version of bokehjs when building our docs

```sh
$ export BOKEH_DOCS_CDN=local
```

We can now go into the sphinx directory where all our docs are stored and build and then locally serve the docs.

```sh
$ cd sphinx
$ make clean all
$ make serve
```

